In [30]:
import time
import datetime
import imageio
import numpy as np
from glob import glob
import os
from natsort import natsorted
import matplotlib.pyplot as plt
import rasterio
from osgeo import gdal
from rasterio.plot import show
import earthpy.plot as ep
import rasterio
from rasterio.enums import Resampling
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

In [48]:
from tensorflow.keras.initializers import glorot_uniform
# from tensorflow.keras.layers import *
# from keras.models import *
# from keras.optimizers import *
# import random
# import mlflow


# https://github.com/keras-team/keras/issues/10074
K.clear_session()

# Use the Sorensen-Dice coefficient as the metric, and the inverse of this as the loss function
# for training the model. See https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient

def sorensen_dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    coef = (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return coef


def sorensen_dice_coef_loss(y_true, y_pred):
    return 1 - sorensen_dice_coef(y_true, y_pred)


def model(input_shape):
    
        model_learning_rate = 0.00001
        model_initializer = glorot_uniform
        kernel_init = model_initializer()
        
        inputs = keras.Input(shape=input_shape)
        conv1 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(inputs)
        conv1 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(conv1)
        pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(pool1)
        conv2 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(conv2)
        pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(pool2)
        conv3 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(conv3)
        pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(pool3)
        conv4 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(conv4)
        drop4 = layers.Dropout(0.01)(conv4)
        pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = layers.Conv2D(1024, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(pool4)
        conv5 = layers.Conv2D(1024, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(conv5)
        drop5 = layers.Dropout(0.01)(conv5)

        up6 = layers.concatenate([layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same', kernel_initializer = kernel_init)(drop5), drop4], axis=3)
        conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer = kernel_init)(up6)
        conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer = kernel_init)(conv6)

        up7 = layers.concatenate([layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', kernel_initializer = kernel_init)(conv6), conv3], axis=3)
        conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer = kernel_init)(up7)
        conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer = kernel_init)(conv7)

        up8 = layers.concatenate([layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', kernel_initializer = kernel_init)(conv7), conv2], axis=3)
        conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer = kernel_init)(up8)
        conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer = kernel_init)(conv8)

        up9 = layers.concatenate([layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', kernel_initializer = kernel_init)(conv8), conv1], axis=3)
        conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer = kernel_init)(up9)
        conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer = kernel_init)(conv9)
        conv9 = layers.Conv2D(2, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = kernel_init)(conv9)

        conv10 = layers.Conv2D(2, (1, 1), activation = 'softmax')(conv9)

        model = keras.Model(inputs=[inputs], outputs=[conv10])

        model.compile(optimizer=Adam(lr=model_learning_rate), loss=sorensen_dice_coef_loss, metrics=['binary_accuracy'])

        return model


In [49]:
unet = model(input_shape=(256, 256, 7))

In [50]:
unet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 7) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 4096        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [51]:
# test new model on a random input image
X = np.random.rand(1, 256, 256, 7)
y_pred = unet.predict(X)
print(y_pred)

[[[[0.5115703  0.48842973]
   [0.52328867 0.4767113 ]
   [0.5307078  0.4692922 ]
   ...
   [0.52260494 0.47739506]
   [0.5281871  0.47181296]
   [0.5102386  0.4897614 ]]

  [[0.5042339  0.4957661 ]
   [0.53148216 0.4685179 ]
   [0.5303799  0.46962008]
   ...
   [0.5242779  0.47572204]
   [0.5399425  0.46005747]
   [0.519853   0.48014703]]

  [[0.50466526 0.4953348 ]
   [0.53953415 0.46046585]
   [0.5345339  0.46546602]
   ...
   [0.5532104  0.44678965]
   [0.522118   0.47788206]
   [0.52904755 0.47095242]]

  ...

  [[0.5227575  0.47724253]
   [0.5330686  0.4669314 ]
   [0.5220007  0.47799933]
   ...
   [0.5150087  0.48499134]
   [0.5421368  0.45786324]
   [0.5151505  0.4848495 ]]

  [[0.5        0.5       ]
   [0.5275365  0.4724635 ]
   [0.5352126  0.4647874 ]
   ...
   [0.5327929  0.46720707]
   [0.5407629  0.45923713]
   [0.5264818  0.47351825]]

  [[0.5037037  0.49629626]
   [0.5153592  0.48464078]
   [0.5152904  0.48470962]
   ...
   [0.5194896  0.4805104 ]
   [0.5174117  0.482588

# Load weights

In [52]:
trained_unet = keras.models.load_model('/Users/cate/Documents/Data_Science_MSc/ECMM433/data/models/mangrove_unet_jean_luc.h5', 
                        custom_objects={'sorensen_dice_coef_loss':sorensen_dice_coef_loss}, compile=True)

In [53]:
trained_unet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 8) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 4672        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [54]:
for i, layer in enumerate(unet.layers):
    try:
        layer.set_weights(trained_unet.layers[i].get_weights())
    except:
        print("Could not transfer weights for layer {}".format(layer.name))

# for layer in unet.layers:
#     try:
#         layer.set_weights(trained_unet.get_layer(name=layer.name).get_weights())
#     except:
#         print("Could not transfer weights for layer {}".format(layer.name))

Could not transfer weights for layer conv2d


In [55]:
new_model = keras.models.model_from_json(unet.to_json())
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 7) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 4096        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [56]:
# X = np.random.rand(1, 256, 256, 3)
y_pred = new_model.predict(X)
print(y_pred)

[[[[0.49976653 0.5002335 ]
   [0.5        0.5       ]
   [0.49840567 0.50159436]
   ...
   [0.5        0.5       ]
   [0.4961345  0.50386554]
   [0.49750268 0.5024973 ]]

  [[0.49908772 0.5009123 ]
   [0.49504265 0.5049574 ]
   [0.49977028 0.5002297 ]
   ...
   [0.48931947 0.51068056]
   [0.4887534  0.5112466 ]
   [0.4914698  0.50853026]]

  [[0.498511   0.501489  ]
   [0.5        0.5       ]
   [0.49135372 0.5086463 ]
   ...
   [0.4851084  0.51489156]
   [0.49443227 0.5055677 ]
   [0.49163577 0.5083642 ]]

  ...

  [[0.49703792 0.5029621 ]
   [0.4938752  0.50612485]
   [0.48531362 0.51468635]
   ...
   [0.4841513  0.5158487 ]
   [0.48789367 0.51210636]
   [0.48973483 0.5102652 ]]

  [[0.5        0.5       ]
   [0.49258757 0.50741243]
   [0.48618782 0.5138122 ]
   ...
   [0.48844844 0.51155156]
   [0.4942047  0.5057953 ]
   [0.48997012 0.5100299 ]]

  [[0.5        0.5       ]
   [0.49023542 0.5097646 ]
   [0.48299184 0.5170081 ]
   ...
   [0.48736268 0.5126373 ]
   [0.49405873 0.505941